In [1]:
%load_ext autoreload

%autoreload 2

import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math

## Finding Lm Ln Polynomials 


In [3]:
def legendre_polynomial(n):
    y = sp.symbols('y')
    if n == 0:
        return sp.Lambda(y, 1)
    elif n == 1:
        return sp.Lambda(y, y)
    else:
        P_n_minus_1 = legendre_polynomial(n-1)
        P_n_minus_2 = legendre_polynomial(n-2)
        return sp.Lambda(y, ((2*n - 1) * y * P_n_minus_1(y) - (n - 1) * P_n_minus_2(y)) / n)

# Example usage:
n = 3
Ln = legendre_polynomial(n)
Ln


Lambda(y, 5*y*(3*y**2/2 - 1/2)/3 - 2*y/3)

## Computing Anm using GL Quadrature --> finding A 

In [33]:
n = [0,1,2]
m = [0,1,2,3]

def Anm(n, m):
    y = sp.symbols('y')
    Ln = legendre_polynomial(n)
    Lm = legendre_polynomial(m)
    return y * Ln(y) * Lm(y)

def Anm_gauleg(xi, ci, b , a, n2, n, m):
    sum = 0 
    Anm_expr = Anm(n,m)
    for i in range(n2):
        y_value = (0.5*(((b-a)*xi[i])+(b+a)))
        prod = ci[i]* Anm_expr.subs(sp.symbols('y'), y_value)
        sum = sum + prod 
    return sum 

def A_matrix(n,m):
    A_matrix = sp.zeros(len(n),len(m))
    n2 = math.floor((len(n)+len(m))/2) + 1
    xi = gl.gauleg(n2)[0] # satisfying dop of GL for p+1 is 2p + 1
    ci = gl.gauleg(n2)[1]
    for ni in n:
        for mi in m:
            A_matrix[ni ,mi] = Anm_gauleg(xi= xi, ci = ci, b = 1, a = -1, n2 = n2, n = ni, m = mi)
    return A_matrix

A = A_matrix(n,m)
A



Matrix([
[                0, 0.666666666666667,                 0, -1.94289029309402e-16],
[0.666666666666667,                 0, 0.266666666666666,                     0],
[                0, 0.266666666666666,                 0,     0.171428571428571]])

## Define A(x,y) 

In [15]:
def a(x, y):
    x = sp.symbols('x')
    y = sp.symbols('y')
    return 5 * x + 1 + y * sp.sin(x)

def a0(x):
    x = sp.symbols('x')
    return 5 * x + 1 

def a1(x):
    x = sp.symbols('x')
    return sp.sin(x)

print(a1(x))

sin(x)


## Define (S1) matrix and (S0) matrix


In [27]:
def GL(xi, ci, y, x, n2, func):
	sum = 0 
	for i in range(n2):
		x_value = 0.5*(((y-x)*xi[i])+(y+x))
		prod = ci[i]*func.subs(sp.symbols('x'), x_value)
		sum = sum + prod 
	return (y-x)/2 * (sum)


def phiij(numofnodes, i, j, l, xlist, func):
	finalsum = 0
	for k in range(numofnodes-1):

		x = xlist[k]
		y = xlist[k+1]
	 
		if k == i :
			d_phi_i = 2**l 
		elif k == i + 1 :
			d_phi_i = -2**l 
		else: 
			d_phi_i = 0 
 
		if k == j : 
			d_phi_j = 2**l 
		elif k == j + 1 :
			d_phi_j = -2**l 
		else:
			d_phi_j = 0 

		n2 = 5 
		xi = gl.gauleg(n2)[0]
		ci = gl.gauleg(n2)[1]
		onegl = GL(xi = xi, ci = ci, y = y, x = x, n2 = 5, func = func) * d_phi_i *d_phi_j
		finalsum = finalsum + onegl

	return finalsum

def S1(listi, listj, func):
    S1 = sp.zeros(len(listi), len(listj))
    target = func
    for i in listi :
        for j in listj:
            S1[i,j] = phiij(numofnodes=2**l +1, i = i, j = j, l = l, xlist = gl.listi(a= 0,b =1, h = 2 **(-l), n1 = 2 **l + 1), func = target)
    return S1


l = 2
ijlist = list(range(2**l-1))
S1(listi = ijlist, listj=ijlist, func= a1(0))


Matrix([
[ 1.95867900975404, -1.46127775712435,                 0],
[-1.46127775712435,  3.79557684538918, -2.33429908826483],
[                0, -2.33429908826483,  5.39648409635573]])

In [30]:
# compute S0 

def S0(listi, listj, func):
    S0 = sp.zeros(len(listi), len(listj))
    target = func
    for i in listi:
        for j in listj:
            S0[i,j] = phiij(numofnodes=2**l +1, i = i, j = j, l = l, xlist = gl.listi(a= 0,b =1, h = 2 **(-l), n1 = 2 **l + 1), func = target)

    return S0 

S0(listi=ijlist, listj=ijlist, func = a0(0))

Matrix([
[ 18.0, -11.5,     0],
[-11.5,  28.0, -16.5],
[    0, -16.5,  38.0]])

## Define Kroneker Notation 


In [38]:
# n, m are integers, probably need to loop it for every possible n, m 

def delta_mn(m,n):
    if n == m:
        return 1 
    else:
        return 0 

def delta(n,m):
    delta = sp.zeros(len(n), len(m))
    for ni in n:
        for mi in m: 
            delta[ni,mi] = delta_mn(ni,mi)

    return delta 

delta(n,m)

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0]])

## Kron Matrix A and S1 

In [36]:
def A_S1(matrixA, matrixS1):
    A_S1 = sp.kronecker_product(matrixA, matrixS1)
    return A_S1

A_S1(matrixA= A_matrix(n= n, m= m), matrixS1= S1(listi=ijlist, listj=ijlist, func = a1(0)))


Matrix([
[                 0,                  0,                 0,   1.30578600650269, -0.974185171416235,                  0,                  0,                  0,                  0, -3.80549843533813e-16,  2.83910236983111e-16,                     0],
[                 0,                  0,                 0, -0.974185171416235,   2.53038456359279,  -1.55619939217655,                  0,                  0,                  0,  2.83910236983111e-16, -7.37438940959908e-16,  4.53528703976797e-16],
[                 0,                  0,                 0,                  0,  -1.55619939217655,   3.59765606423715,                  0,                  0,                  0,                     0,  4.53528703976797e-16, -1.04847765676458e-15],
[  1.30578600650269, -0.974185171416235,                 0,                  0,                  0,                  0,  0.522314402601076, -0.389674068566494,                  0,                     0,                     0,                

## Define S0 * delta_mn


In [40]:
def S0_delta(matrixS0, matrixdelta):
    return matrixS0 * matrixdelta

S0_delta(matrixS0=S0(listi=ijlist, listj=ijlist, func = a0(0)), matrixdelta=delta(n,m))

Matrix([
[ 18.0, -11.5,     0, 0],
[-11.5,  28.0, -16.5, 0],
[    0, -16.5,  38.0, 0]])

## Define Fi and F 